In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve


In [16]:
# --- 1. Load Data ---
# Load the training and testing datasets created in R (Member 6)
try:
    train_df = pd.read_csv('kidney_train.csv')
    test_df = pd.read_csv('kidney_test.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Make sure kidney_train.csv and kidney_test.csv are in the same folder.")

Data loaded successfully.


In [17]:
# --- Separate X and y ---
X_train = train_df.drop('classification', axis=1)
y_train = train_df['classification'].map({'ckd': 1, 'notckd': 0}) # Ensure target is 0/1
X_test = test_df.drop('classification', axis=1)
y_test = test_df['classification'].map({'ckd': 1, 'notckd': 0})


In [20]:
# --- Define Column Types for Preprocessing ---
numeric_features = ['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wbcc','rbcc']
categorical_features = ['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane']

# --- 2. Create Preprocessing Pipeline (FIXED) ---
preprocessor = ColumnTransformer(
    transformers=[
        # FIX 1: Impute Numeric NaN's with the median before scaling.
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        
        # FIX 2: Impute Categorical NaN's with the most frequent value (mode) before encoding.
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features)
    ],
    # Ensure no unlisted column (like the old 'id' column) is causing issues.
    # 'drop' is safer than 'passthrough' if all features are accounted for.
    remainder='drop' 
)

In [21]:
from sklearn.neural_network import MLPClassifier

# --- 3. Build Neural Network Pipeline ---
# Use one hidden layer with 50 neurons and ReLU activation.
# You can experiment with hidden_layer_sizes, e.g., (100, 50) for two layers.
nn_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier(
        hidden_layer_sizes=(50,), # Example: one hidden layer with 50 nodes
        activation='relu',
        solver='adam',
        max_iter=500, # Max iterations for training
        random_state=123,
        # Ensure tolerance is small enough for convergence on this dataset
        tol=1e-4
    ))
])



In [24]:
def evaluate_model(y_true, y_pred, y_proba, model_name):
    print(f"--- {model_name} Performance ---")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred):.4f}")
    print(f"ROC-AUC: {roc_auc_score(y_true, y_proba):.4f}")
    print("---------------------------------")

In [ ]:
# --- 4. Train Model ---
nn_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

In [25]:
# --- 5. Predict and Evaluate ---
y_pred_nn = nn_model.predict(X_test)
y_proba_nn = nn_model.predict_proba(X_test)[:, 1]

evaluate_model(y_test, y_pred_nn, y_proba_nn, "Neural Network (MLPClassifier)")

--- Neural Network (MLPClassifier) Performance ---
Accuracy: 0.9833
Precision: 1.0000
Recall: 0.9750
F1-Score: 0.9873
ROC-AUC: 1.0000
---------------------------------
